# Section 1: CHANGE HERE

In [3]:
# The project folder can be saved in google drive and accessed through Google colab
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
# new language to add, !!!IMPORTANT!!!: the column corresponding to this token
# should have the same column name as in the training dataframe
new_lang_token = 'ch_be'

# Language from which the embeddings should be copied, can be german or
# a dialect the model was trained before
similar_language_token = 'ch_gr' #None #'ch_vs' #deu_Latn' #None  #'ch_gr'

# specifiy if the training data for gr should be limited to the training data of vs
# only relevant when new_lang_token = 'ch_gr'
limit_data = True

# path to the project folder that contains a folder code, data and models
path_project = "/content/drive/MyDrive/anlp_project/NLLB-200/ANLP_SUBMISSION"

# path to the folder where the model is stored or the hugging face model name
model_path = "facebook/nllb-200-distilled-600M"
token_added = None
# when model path is a model that contains token that where added specify which token was added
#model_path = f"{path_project}/models/nllb-de-vs_best_checkpoint"
#token_added = "ch_vs"

# Name and path to save the model
MODEL_SAVE_PATH = f'{path_project}/models/nllb-de-gr-be_initde_v1'

# Training steps
# To train the gr-de model large, set training steps to 5000 as it needs more iterations to converge
training_steps = 2200

# WandB API to log the results, if set to none, no results are logged
api_key = None

# Installations & Imports

In [7]:
!pip install sentencepiece transformers==4.33 datasets wandb sacremoses sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [8]:
import argparse
import logging
import os
import random
import torch
import numpy as np
import pandas as pd
import re
import sys
import typing as tp
import unicodedata
import gc
import random
import sacrebleu
import wandb
import matplotlib.pyplot as plt
from sacremoses import MosesPunctNormalizer
from tqdm.auto import tqdm, trange
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig
from transformers import get_constant_schedule_with_warmup
from transformers.optimization import Adafactor
from sklearn.model_selection import train_test_split

# Data Loading

In [62]:
def prepare_data(data_path, new_lang_token, limit_data):
    """
    Prepare train and validation data.
    """
    df_train = pd.read_csv(f"{data_path}/df_train.csv")
    df_test = pd.read_csv(f"{data_path}/df_test.csv")

    # subset of the data that contains values for this dialect
    df_train = df_train[df_train[['de', new_lang_token]].notna().all(axis=1)].reset_index()
    df_test = df_test[df_test[['de', new_lang_token]].notna().all(axis=1)].reset_index()

    # 2619 vs train
    #print(len(df_train))
    # all dialect exept gr (10475) have between 2700 and 2760 sentences (vs 2619 in train)
    # limit the dataframe for gr to 2619 sentences (train) to make it comparable
    if limit_data and new_lang_token == 'ch_gr':
        df_train = df_train.sample(2619, random_state=42).reset_index(drop=True)

    # split into train and dev
    df_train, df_val = train_test_split(df_train, test_size=0.05, random_state=42)
    print(f"Length train set: {len(df_train)}")
    print(f"Length validation: {len(df_val)}")
    print(f"Length test set: {len(df_test)}")

    return df_train, df_val, df_test

In [63]:
data_path = f"{path_project}/data"

df_train, df_val, df_test = prepare_data(data_path, new_lang_token, limit_data)

Length train set: 2437
Length validation: 129
Length test set: 134


# Prepare the Model and Tokenizer for the new Language Tag

In [13]:
def fix_tokenizer2n(tokenizer, new_lang_tokens):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """

    print(f"BEFORE IDs: {tokenizer.convert_tokens_to_ids(new_lang_tokens[::-1] + ['<mask>'])}")
    print(f"BEFORE Tokens: {tokenizer.convert_ids_to_tokens(tokenizer.convert_tokens_to_ids(new_lang_tokens[::-1] + ['<mask>']))}")


    # how many of the tokens are actually in tokenizer.added_tokens_encoder
    n_added_tokens = len([token for token in new_lang_tokens if token in tokenizer.added_tokens_encoder])

    # get the old/ original length of the tokenizer
    old_len = len(tokenizer) - n_added_tokens

    # move the new tokens in the previous position
    for i, token in enumerate(new_lang_tokens):
      tokenizer.lang_code_to_id[token] = old_len-i
      tokenizer.id_to_lang_code[old_len-old_len-n_added_tokens-i] = token

    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}

    # if the token is not yet trained in the model, add it to the special tokens
    for token in new_lang_tokens:
      if token not in tokenizer._additional_special_tokens:
          tokenizer._additional_special_tokens.append(token)

    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

    print(f"AFTER IDs: {tokenizer.convert_tokens_to_ids(new_lang_tokens[::-1] + ['<mask>'])}")
    print(f"AFTER Tokens: {tokenizer.convert_ids_to_tokens(tokenizer.convert_tokens_to_ids(new_lang_tokens[::-1] + ['<mask>']))}")

def fix_tokenizer1n(tokenizer, new_lang_token):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """
    old_len = len(tokenizer) - int(new_lang_token in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang_token] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang_token
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang_token not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang_token)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

    print(f"Length of the tokenizer: {len(tokenizer)}")
    print(f"IDs: {tokenizer.convert_tokens_to_ids([new_lang_token, '<mask>'])}")
    print(f"IDs: {tokenizer.convert_ids_to_tokens(tokenizer.convert_tokens_to_ids([new_lang_token, '<mask>']))}")

In [14]:
def adapt_embeddings(tokenizer, model, new_lang_token, similar_language_token):

  added_token_id = tokenizer.convert_tokens_to_ids(new_lang_token)
  model.resize_token_embeddings(len(tokenizer))

  # moving the embedding for "mask" to its new position
  model.model.shared.weight.data[added_token_id+1] = model.model.shared.weight.data[added_token_id]

  if similar_language_token is not None:
    similar_lang_id = tokenizer.convert_tokens_to_ids(similar_language_token)
    print(added_token_id, similar_lang_id)
    # initializing new language token with a token of a similar language
    model.model.shared.weight.data[added_token_id] = model.model.shared.weight.data[similar_lang_id]

In [27]:
tokenizer = NllbTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

if model_path=="facebook/nllb-200-distilled-600M":
  # fix tokenizer with new language token
  fix_tokenizer1n(tokenizer, new_lang_token)
else:
  # fix the tokenizer with the added tokens
  fix_tokenizer2n(tokenizer, new_lang_tokens=[new_lang_token, token_added])

# adapt embeddings
adapt_embeddings(tokenizer, model, new_lang_token, similar_language_token)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Length of the tokenizer: 256205
IDs: [256203, 256204]
IDs: ['ch_be', '<mask>']
256203 3


# Training

In [38]:
def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [56]:
# this code is adapted from  the Stopes repo of the NLLB team
# https://github.com/facebookresearch/stopes/blob/main/stopes/pipelines/monolingual/monolingual_line_processor.py#L214

mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]

def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [57]:
LANGS = [('de', 'deu_Latn'), (new_lang_token, new_lang_token)]

def get_batch_pairs(batch_size, data=df_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

In [36]:
# model specific parameters
batch_size = 16
max_length = 128
warmup_steps = 1000
scale_parameter=False
relative_step=False
lr=1e-4
clip_threshold=1.0
weight_decay=1e-3

if api_key is not None:

  project = "nllb-200" # don't need to change
  name = f'{MODEL_SAVE_PATH.split("/")[-1]}'

  !wandb login {api_key}

  # start a new wandb run to track this script
  wandb.init(
      # set the wandb project where this run will be logged
      project=project,
      name=name,
      save_code=True, # save the main script or notebook to W&B
      # track hyperparameters and run metadata
      config={
        "architecture": "nllb-200",
        "dataset": "Swiss-dial",
        "batch_size": batch_size,
        "max_length": max_length,
        "warmup_steps": warmup_steps,
        "training_steps": training_steps,
        "scale_parameter": scale_parameter,
        "relative_step": relative_step,
        "lr": lr,
        "clip_threshold": clip_threshold,
        "weight_decay": weight_decay
      }
  )

In [37]:
model.cuda();

optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=scale_parameter,
    relative_step=relative_step,
    lr=lr,
    clip_threshold=clip_threshold,
    weight_decay=weight_decay,
)

losses = []
val_losses = []
average_train_losses = []
average_val_losses = []
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

In [60]:
# when to save model and printing training and validation loss
save_step = 100

x, y, loss = None, None, None
cleanup()

# to store the best model checkpoint
best_val_loss = float('inf')  # Initialize with a large value
BEST_CHECKPOINT_PATH = f"{MODEL_SAVE_PATH}_best_checkpoint"

tq = trange(len(losses), training_steps)
for i in tq:
    ##################
    ### TRAIN LOOP ###
    ##################
    model.train()
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100
        # Forward Pass and get the loss
        loss = model(**x, labels=y.input_ids).loss
        # backward: perform gradient descent of the loss w.r. to the model params
        loss.backward()
        # Calculate Loss
        losses.append(loss.item())
        # Update Weights, update the model parameters by performing a single optimization step
        optimizer.step()
        # clear the old gradients from optimized variables
        optimizer.zero_grad(set_to_none=True)
        # Update Weights
        scheduler.step()

    except RuntimeError as e:
        # clear the old gradients from optimized variables
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    #######################
    ### VALIDATION LOOP ###
    #######################

    # Set the model to evaluation mode
    model.eval()

    # turn off gradients for validation
    with torch.no_grad():
        xx, yy, lang1, lang2 = get_batch_pairs(batch_size, data=df_val)
        tokenizer.src_lang = lang1
        x_val = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y_val = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        y_val.input_ids[y_val.input_ids == tokenizer.pad_token_id] = -100

        val_loss = model(**x_val, labels=y_val.input_ids).loss

        val_losses.append(val_loss.item())

        # log metrics to wandb
        if api_key is not None:
          wandb.log({"loss-valid": val_loss.item(), "loss-train": loss.item()})

        model.train()

    if i % save_step == 0:
        # get the training loss
        average_train_loss = np.mean(losses[-save_step:])
        average_train_losses.append(average_train_loss)

        # get the validation loss
        average_val_loss = np.mean(val_losses[-save_step:])
        average_val_losses.append(average_val_loss)

        print(i, "| Average train loss: ", average_train_loss, ' | Average validation loss: ', average_val_loss)

        # log metrics to wandb
        if api_key is not None:
          wandb.log({"loss-valid-average": average_val_loss, "loss-train-average": average_train_loss})

        # Check if the current validation loss is the best so far, if yes save the model
        if average_val_loss < best_val_loss and i > 0:
            best_val_loss = average_val_loss
            # Save the best checkpoint
            model.save_pretrained(BEST_CHECKPOINT_PATH)
            tokenizer.save_pretrained(BEST_CHECKPOINT_PATH)
            print(i,f"| Best model saved at {BEST_CHECKPOINT_PATH}")

    # save current model state
    #if i % save_step == 0 and i > 0:
    #    model.save_pretrained(MODEL_SAVE_PATH)
    #    tokenizer.save_pretrained(MODEL_SAVE_PATH)

  0%|          | 0/2198 [00:00<?, ?it/s]

100 | Average train loss:  4.256056859493255  | Average validation loss:  4.697945014238358
100 | Best model saved at /content/drive/MyDrive/anlp_project/NLLB-200/ANLP_SUBMISSION/models/nllb-de-gr-be_initde_best_checkpoint


KeyboardInterrupt: ignored

In [ ]:
if api_key is not None:
  # this is needed in a notebook to finish the logs
  wandb.finish()